# Azure OpenAI Connection Test - System 2

This notebook tests Azure OpenAI connectivity using **two credential modes**:

1. **AWS Secrets Manager Mode (Default)** - Uses boto3 + config.yaml
2. **Environment Variables Mode (Optional)** - Uses .env file

---

## Configuration

Choose your credential mode:

In [1]:
# Set your credential mode here
CREDENTIAL_MODE = "secrets"  # Options: "secrets" or "env"

print(f"Testing with credential mode: {CREDENTIAL_MODE.upper()}")

if CREDENTIAL_MODE == "secrets":
    print("""
📋 AWS Secrets Manager Mode:
  ✅ Credentials fetched from AWS Secrets Manager via boto3
  ✅ Uses config.yaml (cfg.aws['secret_name'], cfg.aws['region_name'])
  ✅ Requires: AWS credentials (AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
  ✅ Requires: boto3 installed
    """)
else:
    print("""
📋 Environment Variables Mode:
  ✅ Credentials from .env file
  ✅ Requires: AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_KEY
    """)

Testing with credential mode: SECRETS

📋 AWS Secrets Manager Mode:
  ✅ Credentials fetched from AWS Secrets Manager via boto3
  ✅ Uses config.yaml (cfg.aws['secret_name'], cfg.aws['region_name'])
  ✅ Requires: AWS credentials (AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
  ✅ Requires: boto3 installed
    


---

## Setup

In [2]:
import os
import sys
import json
from pathlib import Path
from dotenv import load_dotenv

project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

load_dotenv()

print("✅ Environment setup complete")
print(f"Project root: {project_root}")

✅ Environment setup complete
Project root: /home/sagemaker-user/SP_Streamlit


In [3]:
from shared.utils.utils import gai, initialize_client, get_secret, Config

print("✅ Imports successful")

✅ Imports successful


---

## Test 1: Configuration Check

In [4]:
print("="*80)
print(f"TEST 1: Configuration Check ({CREDENTIAL_MODE.upper()} Mode)")
print("="*80)

if CREDENTIAL_MODE == "secrets":
    # AWS Secrets Manager mode - check config.yaml and AWS credentials
    print("\nMode: AWS Secrets Manager (boto3)")
    print("Credentials will be fetched from AWS Secrets Manager, NOT environment variables\n")
    
    try:
        # Load config.yaml
        cfg = Config.from_yaml('shared/config/config.yaml')
        print(f"✅ config.yaml loaded")
        
        # Check AWS config section
        if hasattr(cfg, 'aws'):
            print(f"\n✅ AWS configuration found in config.yaml:")
            print(f"  - secret_name: {cfg.aws.get('secret_name', 'NOT SET')}")
            print(f"  - region_name: {cfg.aws.get('region_name', 'NOT SET')}")
            print(f"  - api_version: {cfg.aws.get('api_version', 'NOT SET')}")
        else:
            print(f"\n❌ No 'aws' section in config.yaml")
            raise ValueError("config.yaml missing 'aws' section")
        
        # Check AWS credentials (for boto3)
        aws_key = os.getenv('AWS_ACCESS_KEY_ID')
        aws_secret = os.getenv('AWS_SECRET_ACCESS_KEY')
        
        if aws_key and aws_secret:
            print(f"\n✅ AWS credentials found (for boto3):")
            print(f"  - AWS_ACCESS_KEY_ID: {aws_key[:8]}...{aws_key[-4:]}")
            print(f"  - AWS_SECRET_ACCESS_KEY: ****")
        else:
            print(f"\n❌ AWS credentials not found")
            print(f"  boto3 needs AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY")
            raise ValueError("Missing AWS credentials for boto3")
        
        print(f"\n✅ TEST 1 PASSED")
        print(f"\n⚠️  Note: Azure credentials will be fetched from AWS Secrets Manager when needed")
        
    except Exception as e:
        print(f"\n❌ TEST 1 FAILED")
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

else:
    # Environment variables mode - check AZURE_OPENAI_* env vars
    print("\nMode: Environment Variables")
    print("Checking for AZURE_OPENAI_* environment variables\n")
    
    required_vars = {
        'AZURE_OPENAI_ENDPOINT': os.getenv('AZURE_OPENAI_ENDPOINT'),
        'AZURE_OPENAI_API_KEY': os.getenv('AZURE_OPENAI_API_KEY'),
    }
    
    all_set = True
    for var_name, var_value in required_vars.items():
        if var_value:
            if 'KEY' in var_name:
                masked = var_value[:8] + '...' + var_value[-4:] if len(var_value) > 12 else '***'
                print(f"  ✅ {var_name}: {masked}")
            else:
                print(f"  ✅ {var_name}: {var_value}")
        else:
            print(f"  ❌ {var_name}: NOT SET")
            all_set = False
    
    optional_vars = {
        'AZURE_OPENAI_API_VERSION': os.getenv('AZURE_OPENAI_API_VERSION', '2024-02-15-preview'),
        'AZURE_OPENAI_DEPLOYMENT': os.getenv('AZURE_OPENAI_DEPLOYMENT', 'gpt-4o-mini'),
    }
    
    print("\nOptional (with defaults):")
    for var_name, var_value in optional_vars.items():
        print(f"  {var_name}: {var_value}")
    
    if all_set:
        print(f"\n✅ TEST 1 PASSED")
    else:
        print(f"\n❌ TEST 1 FAILED: Missing required environment variables")

TEST 1: Configuration Check (SECRETS Mode)

Mode: AWS Secrets Manager (boto3)
Credentials will be fetched from AWS Secrets Manager, NOT environment variables

✅ config.yaml loaded

✅ AWS configuration found in config.yaml:
  - secret_name: AZURE_OPENAI_API_KEY_NORTHCENTRALUS
  - region_name: us-east-1
  - api_version: 2024-08-01-preview

❌ AWS credentials not found
  boto3 needs AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY

❌ TEST 1 FAILED
Error: Missing AWS credentials for boto3


Traceback (most recent call last):
  File "/tmp/ipykernel_10598/1752138901.py", line 36, in <module>
    raise ValueError("Missing AWS credentials for boto3")
ValueError: Missing AWS credentials for boto3


---

## Test 2: Fetch Secret from AWS Secrets Manager

**Only for AWS Secrets Manager mode** - test that we can fetch the secret via boto3.

In [5]:
if CREDENTIAL_MODE == "secrets":
    print("="*80)
    print("TEST 2: Fetch Secret from AWS Secrets Manager")
    print("="*80)
    
    try:
        print("\nAttempting to fetch secret from AWS Secrets Manager...")
        print(f"Using boto3 with config from config.yaml\n")
        
        secret_string = get_secret()
        credentials = json.loads(secret_string)
        
        print(f"✅ Secret fetched successfully from AWS Secrets Manager")
        print(f"\nSecret contains:")
        for key in credentials.keys():
            if key.lower() in ['key', 'api_key', 'apikey']:
                print(f"  - {key}: ****")
            elif key.lower() in ['endpoint', 'url']:
                print(f"  - {key}: {credentials[key]}")
            else:
                print(f"  - {key}: {credentials[key]}")
        
        # Verify expected fields
        if 'endpoint' in credentials and 'key' in credentials:
            print(f"\n✅ TEST 2 PASSED: Secret has required fields (endpoint, key)")
        else:
            print(f"\n⚠️  TEST 2 WARNING: Secret missing 'endpoint' or 'key' fields")
            print(f"  Expected format: {{\"endpoint\": \"...\", \"key\": \"...\"}}")
        
    except Exception as e:
        print(f"\n❌ TEST 2 FAILED")
        print(f"Error: {e}")
        print(f"\n💡 Troubleshooting:")
        print(f"  - Verify the secret exists in AWS Secrets Manager")
        print(f"  - Check secret name matches config.yaml")
        print(f"  - Ensure IAM permissions allow reading the secret")
        import traceback
        traceback.print_exc()
else:
    print("Skipping Test 2 (only for AWS Secrets Manager mode)")

TEST 2: Fetch Secret from AWS Secrets Manager

Attempting to fetch secret from AWS Secrets Manager...
Using boto3 with config from config.yaml

✅ Secret fetched successfully from AWS Secrets Manager

Secret contains:
  - key: ****
  - endpoint: https://cog-mele-services-dev-openai-northcentralus.openai.azure.com/
  - region: northcentralus

✅ TEST 2 PASSED: Secret has required fields (endpoint, key)


---

## Test 3: Azure OpenAI Client Initialization

In [6]:
print("="*80)
print(f"TEST 3: Client Initialization ({CREDENTIAL_MODE.upper()} Mode)")
print("="*80)

use_env_vars = (CREDENTIAL_MODE == "env")

try:
    print(f"\nInitializing Azure OpenAI client...")
    if use_env_vars:
        print(f"Credential source: Environment Variables")
    else:
        print(f"Credential source: AWS Secrets Manager (via boto3)")
    
    client = initialize_client(use_env_vars=use_env_vars)
    
    print(f"\n✅ Client initialized successfully")
    print(f"Client type: {type(client).__name__}")
    print(f"\n✅ TEST 3 PASSED")
    
except Exception as e:
    print(f"\n❌ TEST 3 FAILED")
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

TEST 3: Client Initialization (SECRETS Mode)

Initializing Azure OpenAI client...
Credential source: AWS Secrets Manager (via boto3)

✅ Client initialized successfully
Client type: AzureOpenAI

✅ TEST 3 PASSED


---

## Test 4: Simple LLM Call

In [7]:
print("="*80)
print(f"TEST 4: Simple LLM Call ({CREDENTIAL_MODE.upper()} Mode)")
print("="*80)

use_env = (CREDENTIAL_MODE == "env")

try:
    print(f"\nSending test prompt to Azure OpenAI...")
    
    response = gai(
        sys_prompt="You are a helpful assistant.",
        user_prompt="Say 'Hello from Azure OpenAI!' and nothing else.",
        model="gpt-4o-mini",
        source="azure",
        azure_use_env=use_env
    )
    
    print(f"\n✅ LLM call successful!")
    print(f"Response: {response}")
    print(f"\n✅ TEST 4 PASSED")
    
except Exception as e:
    print(f"\n❌ TEST 4 FAILED")
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

TEST 4: Simple LLM Call (SECRETS Mode)

Sending test prompt to Azure OpenAI...
  [AZURE] Calling Azure OpenAI (credentials from AWS Secrets Manager)

✅ LLM call successful!
Response: Hello from Azure OpenAI!

✅ TEST 4 PASSED


---

## Test 5: JSON Response Parsing

In [8]:
print("="*80)
print(f"TEST 5: JSON Response Parsing ({CREDENTIAL_MODE.upper()} Mode)")
print("="*80)

use_env = (CREDENTIAL_MODE == "env")

try:
    print(f"\nRequesting JSON response...")
    
    response = gai(
        sys_prompt="You are a helpful assistant that responds in JSON format.",
        user_prompt=(
            "Respond with a JSON object with these fields:\n"
            "- status: 'success'\n"
            "- message: 'Azure OpenAI is working'\n"
            "- test_passed: true\n"
            "Respond ONLY with the JSON object."
        ),
        model="gpt-4o-mini",
        source="azure",
        azure_use_env=use_env
    )
    
    if isinstance(response, dict):
        print(f"\n✅ JSON parsed successfully:")
        for key, value in response.items():
            print(f"  - {key}: {value}")
        
        if response.get('status') == 'success':
            print(f"\n✅ TEST 5 PASSED")
        else:
            print(f"\n⚠️  JSON received but validation failed")
    else:
        print(f"\n⚠️  Response not parsed as JSON: {response}")
        print(f"❌ TEST 5 FAILED")
    
except Exception as e:
    print(f"\n❌ TEST 5 FAILED")
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

TEST 5: JSON Response Parsing (SECRETS Mode)

Requesting JSON response...
  [AZURE] Calling Azure OpenAI (credentials from AWS Secrets Manager)

✅ JSON parsed successfully:
  - status: success
  - message: Azure OpenAI is working
  - test_passed: True

✅ TEST 5 PASSED


---

## Test 6: Materiality Scoring Format

In [9]:
print("="*80)
print(f"TEST 6: Materiality Scoring Format ({CREDENTIAL_MODE.upper()} Mode)")
print("="*80)

use_env = (CREDENTIAL_MODE == "env")

sys_prompt = """You are an expert analyst assessing the material impact of soft power events.

Assign a materiality score from 1.0 to 10.0.

Scoring Scale:
- 1-3: Symbolic/rhetorical
- 4-6: Mixed symbolic and material
- 7-10: Highly material (concrete infrastructure, specific financial commitments)

Output JSON format:
{"material_score": 7.5, "justification": "Brief explanation"}

Respond with ONLY the JSON object."""

user_prompt = """Event: China-Egypt Belt and Road Infrastructure Agreement
Country: China | Recipients: Egypt | Date: 2024-08-15

Description: China and Egypt signed infrastructure agreement under Belt and Road Initiative, 
including $2.5 billion financing for high-speed rail and port expansion. Construction starts Q1 2025."""

try:
    print(f"\nTesting materiality scoring format...")
    
    response = gai(
        sys_prompt=sys_prompt,
        user_prompt=user_prompt,
        model="gpt-4o-mini",
        source="azure",
        azure_use_env=use_env
    )
    
    if isinstance(response, dict):
        score = response.get('material_score')
        justification = response.get('justification', '')
        
        print(f"\n✅ Materiality response:")
        print(f"  - Material Score: {score}/10.0")
        print(f"  - Justification: {justification[:100]}..." if len(justification) > 100 else f"  - Justification: {justification}")
        
        if isinstance(score, (int, float)) and 1.0 <= score <= 10.0:
            print(f"\n✅ TEST 6 PASSED")
            print(f"\n🎉 Azure OpenAI setup is ready for the materiality scoring pipeline!")
        else:
            print(f"\n❌ Score validation failed (expected 1.0-10.0, got {score})")
    else:
        print(f"\n⚠️  Response not JSON: {response}")
        print(f"❌ TEST 6 FAILED")
    
except Exception as e:
    print(f"\n❌ TEST 6 FAILED")
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

TEST 6: Materiality Scoring Format (SECRETS Mode)

Testing materiality scoring format...
  [AZURE] Calling Azure OpenAI (credentials from AWS Secrets Manager)

✅ Materiality response:
  - Material Score: 9.0/10.0
  - Justification: The event involves a large, specific financial commitment ($2.5 billion) for tangible infrastructure...

✅ TEST 6 PASSED

🎉 Azure OpenAI setup is ready for the materiality scoring pipeline!


---

## Summary

In [10]:
print("="*80)
print("SUMMARY")
print("="*80)

if CREDENTIAL_MODE == "secrets":
    print("""
✅ AWS Secrets Manager Mode (Production)

If all tests passed, you're ready to use:

  response = gai(
      sys_prompt="...",
      user_prompt="...",
      source="azure"  # Uses boto3 + AWS Secrets Manager automatically
  )

What happens:
  1. gai() calls initialize_client(use_env_vars=False)
  2. initialize_client() calls get_secret()
  3. get_secret() reads cfg.aws from config.yaml
  4. boto3 fetches secret from AWS Secrets Manager
  5. Azure credentials extracted and used

No Azure environment variables needed!
    """)
else:
    print("""
✅ Environment Variables Mode (Alternative)

If all tests passed, you're ready to use:

  response = gai(
      sys_prompt="...",
      user_prompt="...",
      source="azure",
      azure_use_env=True  # Use env vars instead of AWS Secrets Manager
  )
    """)

SUMMARY

✅ AWS Secrets Manager Mode (Production)

If all tests passed, you're ready to use:

  response = gai(
      sys_prompt="...",
      user_prompt="...",
      source="azure"  # Uses boto3 + AWS Secrets Manager automatically
  )

What happens:
  1. gai() calls initialize_client(use_env_vars=False)
  2. initialize_client() calls get_secret()
  3. get_secret() reads cfg.aws from config.yaml
  4. boto3 fetches secret from AWS Secrets Manager
  5. Azure credentials extracted and used

No Azure environment variables needed!
    
